**Script Description:** This script calculates key soil hydraulic properties (Wilting Point, Saturation, Porosity) using the BOFEK2020 methodology and source data. It loads site data containing BOFEK2020 unit assignments, maps these units to their dominant soil profiles using the official BOFEK2020 results table, retrieves the layer structure for these profiles, looks up the corresponding Staringreeks 2018 Mualem-Van Genuchten parameters, calculates the hydraulic properties per layer, and finally aggregates or selects these properties based on a user-defined target depth or interval.

**File Name:** 01_06_Extract_BOFEK_Data.ipynb

**Date:** 2025

**Created by:** Rob Alamgir

**References:**

   [1] Heinen, M., Brouwer, F., Teuling, K., Walvoort, D. (2021).
       BOFEK2020 - Soil physical schematization of the Netherlands; Update soil physical unit map.
       Wageningen Environmental Research, Report 3056.
       DOI: https://doi.org/10.18174/541544     [cite: 1]

   [2] Heinen, M., Bakker, G., Wösten, H. (2020).
       Water retention and permeability characteristics of tops and subsurfaces in the Netherlands: the Staring series; update 2018.
       Wageningen Environmental Research, Report 2978.
       (Referenced in BOFEK2020 report [cite: 1, page 36])

   [3] BOFEK2020 Data Download:
       Wageningen University & Research. Bodemfysische Eenhedenkaart (BOFEK2020).
       Retrieved from: https://www.wur.nl/nl/show/Bodemphysical-Eenhedenkaart-BOFEK2020.htm [cite: 1, pages 10, 34]

   [4] Van Genuchten, M.Th. (1980).
       A closed-form equation for predicting the hydraulic conductivity of unsaturated soils.
       Soil Science Society of America Journal 44(3): 892-898.
       (Referenced in BOFEK2020 report [cite: 1, page 36])

   [5] Mualem, Y. (1976).
       A new model for predicting the hydraulic conductivity of unsaturated porous media.
       Water Resources Research, 12: 513-522.
       (Referenced in BOFEK2020 report [cite: 1, page 36])

#### Import the relevant packages

In [1]:
import pandas as pd
import numpy as np

#### Import all the relevant datasets

In [2]:
NOBV_Soil_Data_CSV_data_path = "C:/Data_MSc_Thesis/NOBV_Site_Data/NOBV_Soil_Data_CSV.csv"
NOBV_Soil_Data_CSV = pd.read_csv(NOBV_Soil_Data_CSV_data_path, encoding='ISO-8859-1')
#NOBV_Soil_Data_CSV.info(verbose=True)
#NOBV_Soil_Data_CSV.head(22)

all_results_formatted_95_data_path = "C:/Data_MSc_Thesis/BOFEK/3_Clustering/Output/all_results_formatted_95.csv"
all_results_formatted_95 = pd.read_csv(all_results_formatted_95_data_path)
#all_results_formatted_95.info(verbose=True)
#all_results_formatted_95.head(10)

AllProfiles_368_data_path = "C:/Data_MSc_Thesis/BOFEK/1_CalcAllPars/Data/AllProfiles_368.csv"
AllProfiles_368 = pd.read_csv(AllProfiles_368_data_path)
#AllProfiles_368.info(verbose=True)
#AllProfiles_368.head(10)

StaringReeksPARS_2018_data_path = "C:/Data_MSc_Thesis/BOFEK/1_CalcAllPars/Data/StaringReeksPARS_2018.csv"
StaringReeksPARS_2018 = pd.read_csv(StaringReeksPARS_2018_data_path)
#StaringReeksPARS_2018.info(verbose=True)
#StaringReeksPARS_2018.head(10)

In [ ]:
# Load and preprocess data
#data_path = "C:/Data_MSc_Thesis/BOFEK/1_CalcAllPars/Output/OUT_AllProfiles_ALL.csv"
#OUT_AllProfiles_ALL = pd.read_csv(data_path)
##OUT_AllProfiles_ALL.info(verbose=True)
#OUT_AllProfiles_ALL.head(10)

# Load and preprocess data
#data_path = "C:/Data_MSc_Thesis/BOFEK/1_CalcAllPars/Output/profile_info.csv"
#profile_info = pd.read_csv(data_path)
##profile_info.info(verbose=True)
#profile_info.head(10)

# Load and preprocess data
#data_path = "C:/Data_MSc_Thesis/BOFEK/3_Clustering/Data/BodemCode.csv"
#BodemCode = pd.read_csv(data_path)
##BodemCode.info(verbose=True)
#BodemCode.head(10)

#### Specify DataFrame variables 

In [3]:
sites_df = NOBV_Soil_Data_CSV                # DataFrame from your site list CSV
bofek_results_df = all_results_formatted_95  # DataFrame from all_results_formatted_95.csv
profiles_df = AllProfiles_368                # DataFrame from AllProfiles_368.csv
staring_df = StaringReeksPARS_2018           # DataFrame from StaringReeksPARS_2018.csv

# From NOBV_Soil_Data_CSV
site_id_col = 'Site_ID'
bofek_unit_col = 'BOFEK_2020_Physical Units'

# From all_results_formatted_95 
bofek_profile_id_col = 'iProfile'
bofek_cluster_col = 'clust1'    # BOFEK Unit number
bofek_dominant_col = 'dominant' # Flag for dominant profile (value=1)

profiles_profile_id_col = 'iProfile' # From AllProfiles_368 

# From StaringReeksPARS_2018
staring_id_col = 'StaringID'
staring_theta_r_col = 'theta_r'
staring_theta_s_col = 'theta_s'
staring_alpha_col = 'alpha'
staring_n_col = 'n'

# Define Constants
target_depth_cm = 5.0  # Target Depth
use_interval = False
print(f"Calculating for specific depth: {target_depth_cm} cm")
h_wp = -16000.0        # Pressure head for Wilting Point in cm [cite: 1, page 17]

Calculating for specific depth: 5.0 cm


### Step 1: Rename Staring dataframe Columns (Matching .head() output)

In [4]:
# Use the exact column names from your StaringReeksPARS_2018.head() output
# as the keys in the dictionary, mapping them to the target names used later.
try:
    staring_df.rename(columns={
        'i': staring_id_col,        # Original column 'i'
        'WCr': staring_theta_r_col, # Original column 'WCr'
        'WCs': staring_theta_s_col, # Original column 'WCs'
        'Alpha': staring_alpha_col, # Original column 'Alpha'
        'Npar': staring_n_col       # Original column 'Npar'
        # Add 'Lambda': 'lambda', 'Ksfit': 'ksat' if needed
    }, inplace=True, errors='raise') # Use 'raise' to catch errors if columns don't exist
    print("Renamed Staring DF columns successfully.")
except KeyError as e:
    print(f"Error renaming Staring DF columns: Column '{e}' not found.")
    print("Please ensure staring_df is loaded and original column names are exactly 'i', 'WCr', 'WCs', 'Alpha', 'Npar'.")

Renamed Staring DF columns successfully.


### Step 2: Create BOFEK Unit -> Dominant Profile Map

In [5]:
dominant_profile_map = {} # Initialize empty map
try:
    # Filter bofek_results_df to get only the dominant profiles (dominant == 1)
    dominant_profiles = bofek_results_df[bofek_results_df[bofek_dominant_col] == 1].copy()

    # Create the mapping dictionary {BOFEK_Unit : dominant_iProfile}
    dominant_profile_map = pd.Series(
        dominant_profiles[bofek_profile_id_col].values,
        index=dominant_profiles[bofek_cluster_col] # BOFEK unit number
    ).to_dict()
    print(f"Created map for {len(dominant_profile_map)} dominant profiles.")
    if not dominant_profile_map:
         print("Warning: Dominant profile map is empty. Check 'dominant' column in bofek_results_df.")
except KeyError as e:
    print(f"Error creating dominant profile map: Column '{e}' not found in bofek_results_df.")
    print("Please ensure columns are named correctly ('iProfile', 'clust1', 'dominant').")
except Exception as e:
    print(f"An unexpected error occurred creating the dominant profile map: {e}")

Created map for 79 dominant profiles.


### Step 3: Add Dominant Profile ID to the Site Data

In [6]:
# Define the new column name clearly
dominant_profile_id_col = 'Dominant_iProfile' # Explicit name for clarity
if dominant_profile_map:                      # Proceed only if map was created
    sites_df[dominant_profile_id_col] = sites_df[bofek_unit_col].map(dominant_profile_map)
    # Check for mapping issues
    if sites_df[dominant_profile_id_col].isnull().any():
        print("\nWarning: Some sites could not be mapped to a dominant profile!")
        print("Check the BOFEK units in your site data and the dominant profile map.")
        # Show only sites with missing dominant profile IDs
        print(sites_df[sites_df[dominant_profile_id_col].isnull()][[site_id_col, bofek_unit_col]])
else:
    print("\nError: Cannot map dominant profiles. Dominant profile map is empty or invalid.")
    sites_df[dominant_profile_id_col] = np.nan # Add column but fill with NaN

### Step 4: Define Calculation Function

$$ \theta(h) = \theta_r + \frac{\theta_s - \theta_r}{ \left[ 1 + (|\alpha h|)^n \right]^{1 - 1/n} } $$

Reminder: This formula applies when the soil is unsaturated (h < 0). If the soil is saturated (h ≥ 0), the formula simplifies to: θ(h) = θs

In [7]:
# pressure head (h)
# soil water content (theta)
# saturated water content (theta_s , θs)
# residual water content (theta_r, θr)
# van Genuchten parameters (alpha, n) as input.
# α (alpha)  van Genuchten parameter related to air entry pressure (units of 1/Length).
# n is the van Genuchten parameter related to pore size distribution (dimensionless, n > 1).

def calculate_theta(h, theta_s, theta_r, alpha, n):
    """Calculates water content theta using Mualem-Van Genuchten."""
    
    # Input validation by checking if any of the input values are NaN
    if pd.isna(h) or pd.isna(theta_s) or pd.isna(theta_r) or pd.isna(alpha) or pd.isna(n):
        return np.nan
    
    # parameter n must be greater than 1 for the model to be physically realistic and 
    # mathematically well-behaved (specifically, it ensures m = 1 - 1/n is positive). Returns NaN if n is invalid.   
    if n <= 1.0:
        return np.nan 
    
    # saturated water content (theta_s) cannot be less than residual water content, physically impossible  
    if theta_s < theta_r:
        return np.nan 

    # Handles the saturated or positive pressure case. If the pressure head h is zero or positive, 
    # the soil is assumed fully saturated, and the water content is simply theta_s.
    if h >= 0:
        return theta_s 
    
    # Calculates the dependent parameter m based on n, according to the standard constraint often used with the Mualem model.
    m = 1.0 - (1.0 / n)
    
    # Mathematically, if alpha is 0, the term (|αh|)ⁿ is 0 (for h<0), the denominator [1+0]ᵐ is 1, and theta should equal theta_s.
    if alpha == 0:
        return theta_r # Limit case as pressure becomes very negative

    try:
        # block to catch potential numerical errors (like numbers becoming too large or small, or invalid operations)
        
        # Calculates the base term |αh|. np.abs ensures it's positive since h is negative in this part of the code.
        term_base = np.abs(alpha * h)
        # Overflow Prevention: This cleverly checks if calculating term_base**n would likely result in an overflow error 
        # before actually doing the calculation. np.exp(700) is roughly the largest number a standard float can hold. 
        # If log(term_base^n) (which equals n * log(term_base)) exceeds 700, the result of the exponentiation would be too large. 
        # In this limit (corresponding to very large negative h, i.e., very dry soil), theta approaches theta_r.
        if n * np.log(term_base) > 700:
             return theta_r
        
        # Calculates 1 + (|αh|)ⁿ. It then checks if the result (term) is finite and positive.
        term = 1.0 + np.power(term_base, n)
        if not np.isfinite(term) or term <= 0:
            return np.nan

        # Check power m
        if m * np.log(term) < -700: # Avoid underflow leading to division by zero
             return theta_s # Denominator is ~0, fraction is huge, approaches theta_s (incorrect limit?) -> better NaN? Let's try NaN.
             # Actually, if denominator -> 0, term is huge, theta -> theta_r.
             # Let's rethink: If term**m -> 0, then theta -> theta_r + (theta_s - theta_r) / 0 -> Inf?
             # If h-> -inf, term_base -> inf, term -> inf, term**m -> inf (assuming m>0), denom -> inf, frac -> 0, theta -> theta_r.
             # Let's trust the formula limit. If term**m results in 0 or non-finite, it's likely an issue.
             return theta_r # If denominator is effectively zero due to underflow
            
        # Calculates the full denominator [ 1 + (|αh|)ⁿ ]ᵐ. 
        # Checks again if the result is finite and non-zero to prevent division by zero.
        denominator = np.power(term, m)
        if not np.isfinite(denominator) or denominator == 0:
            return np.nan 

        # Calculates the final theta value using the rearranged van Genuchten formula.
        theta = theta_r + (theta_s - theta_r) / denominator

        # Final Bounds Check: Checks if the calculated theta is within the physical bounds (theta_r to theta_s), 
        # allowing for a tiny numerical tolerance (1e-6). Instead of forcing the value into the bounds (clamping), 
        # it returns NaN, indicating that something potentially went wrong in the calculation if the result is outside the expected physical range.
        if theta < theta_r - 1e-6 or theta > theta_s + 1e-6:
            # print(f"  Debug: Calculated theta {theta} outside bounds ({theta_r=}, {theta_s=})")
            return np.nan
        return theta # If all calculations are successful and checks pass, the final calculated theta value is returned.

    except (OverflowError, ValueError, RuntimeWarning) as e: # Catch more numeric issues
        # print(f"  Debug: Numeric exception in calculate_theta ({e})")
        return np.nan

### Step 5: Process Each Site

In [8]:
results = []                  # Creates an empty list to store the final calculated properties for each site.
processed_profiles_cache = {} # Cache results {profile_id: {'WP': wp, 'SAT': sat, 'Porosity': poro}}

print("\n--- Processing Sites ---")
for index, site_row in sites_df.iterrows():
    site_id = site_row[site_id_col]
    profile_id = site_row[dominant_profile_id_col] # Use the mapped dominant profile ID

    # Initialize results for this site
    site_result_wp = np.nan
    site_result_sat = np.nan
    site_result_porosity = np.nan

    if pd.isna(profile_id):
        print(f"Skipping site {site_id}: No dominant profile mapped.")
        results.append({'Site_ID': site_id, 'WP': site_result_wp, 'SAT': site_result_sat, 'Porosity': site_result_porosity})
        continue

    try:
        profile_id_int = int(profile_id)
    except ValueError:
        print(f"Skipping site {site_id}: Invalid dominant profile ID '{profile_id}'.")
        results.append({'Site_ID': site_id, 'WP': site_result_wp, 'SAT': site_result_sat, 'Porosity': site_result_porosity})
        continue

    # Check cache first
    if profile_id_int in processed_profiles_cache:
        cached_result = processed_profiles_cache[profile_id_int]
        site_result_wp = cached_result['WP']
        site_result_sat = cached_result['SAT']
        site_result_porosity = cached_result['Porosity']
        print(f"Processing Site: {site_id} (Profile: {profile_id_int}) - Using cached results")
        results.append({'Site_ID': site_id, 'WP': site_result_wp, 'SAT': site_result_sat, 'Porosity': site_result_porosity})
        continue # Move to next site

    # Process Profile (if not cached) 
    print(f"Processing Site: {site_id} (Profile: {profile_id_int}) - Calculating...")
    profile_structure = profiles_df[profiles_df[profiles_profile_id_col] == profile_id_int]

    if profile_structure.empty:
        print(f"  Error: Profile structure for iProfile {profile_id_int} not found.")
        # Store NaN in cache for this profile ID to avoid repeated lookups
        processed_profiles_cache[profile_id_int] = {'WP': np.nan, 'SAT': np.nan, 'Porosity': np.nan}
        results.append({'Site_ID': site_id, 'WP': site_result_wp, 'SAT': site_result_sat, 'Porosity': site_result_porosity})
        continue

    # Extract layers and calculate layer properties 
    layer_results_list = []
    last_depth = 0.0
    valid_layers_found_structure = False
    for i in range(1, 10): # Max 9 layers
        soil_col = f'iSoil{i}'
        depth_col = f'iZ{i}'
        if soil_col not in profile_structure.columns or depth_col not in profile_structure.columns: break

        staring_id_val = profile_structure.iloc[0][soil_col]
        bottom_depth_val = profile_structure.iloc[0][depth_col]

        # Check for termination conditions or NaN Staring ID
        if pd.isna(staring_id_val) or staring_id_val <= 0 or bottom_depth_val > 9999: break

        top_depth_val = last_depth
        thickness_val = bottom_depth_val - top_depth_val

        if thickness_val <= 1e-6: # Use tolerance for thickness check
             # print(f"  Warning: Near-zero layer thickness ({thickness_val}cm) for layer {i} in profile {profile_id_int}.")
             last_depth = bottom_depth_val # Still update depth
             continue

        valid_layers_found_structure = True
        current_staring_id = int(staring_id_val)

        # Lookup parameters
        params = staring_df[staring_df[staring_id_col] == current_staring_id]
        if params.empty:
            print(f"    Warning: Parameters for StaringID {current_staring_id} (Layer {i}) not found.")
            layer_wp_val, layer_sat_val, layer_porosity_val = np.nan, np.nan, np.nan
        else:
            param_row = params.iloc[0]
            theta_s = param_row[staring_theta_s_col]
            theta_r = param_row[staring_theta_r_col]
            alpha = param_row[staring_alpha_col]
            n = param_row[staring_n_col]

            layer_sat_val = theta_s
            layer_porosity_val = theta_s # Porosity = theta_s
            layer_wp_val = calculate_theta(h_wp, theta_s, theta_r, alpha, n)

        layer_results_list.append({
            'WP': layer_wp_val, 'SAT': layer_sat_val, 'Porosity': layer_porosity_val,
            'Top': top_depth_val, 'Bottom': bottom_depth_val, 'Thickness': thickness_val
        })
        last_depth = bottom_depth_val
        if last_depth >= 120.0 - 1e-6: break # Stop if profile depth reaches ~120 cm

    # Aggregate/Select for Target Depth 
    if not valid_layers_found_structure:
        print(f"  Error: No valid layers extracted for profile {profile_id_int}.")
        # Store NaN in cache
        processed_profiles_cache[profile_id_int] = {'WP': np.nan, 'SAT': np.nan, 'Porosity': np.nan}
        results.append({'Site_ID': site_id, 'WP': np.nan, 'SAT': np.nan, 'Porosity': np.nan})
        continue

    # Perform aggregation/selection based on layer_results_list
    if use_interval:
        interval_top, interval_bottom = target_interval_cm
        total_thickness_in_interval = 0.0
        weighted_wp_sum = 0.0
        weighted_sat_sum = 0.0
        weighted_porosity_sum = 0.0
        valid_data_found = False
        for res in layer_results_list:
            overlap_top = max(res['Top'], interval_top)
            overlap_bottom = min(res['Bottom'], interval_bottom)
            overlap_thickness = max(0.0, overlap_bottom - overlap_top)
            if overlap_thickness > 1e-6:
                if not pd.isna(res['WP']) and not pd.isna(res['SAT']) and not pd.isna(res['Porosity']):
                    total_thickness_in_interval += overlap_thickness
                    weighted_wp_sum += res['WP'] * overlap_thickness
                    weighted_sat_sum += res['SAT'] * overlap_thickness
                    weighted_porosity_sum += res['Porosity'] * overlap_thickness
                    valid_data_found = True
        if valid_data_found and total_thickness_in_interval > 1e-6:
            site_result_wp = weighted_wp_sum / total_thickness_in_interval
            site_result_sat = weighted_sat_sum / total_thickness_in_interval
            site_result_porosity = weighted_porosity_sum / total_thickness_in_interval
        else:
            print(f"  Warning: No valid layers with data found within target interval {target_interval_cm} for profile {profile_id_int}")
    else: # Specific depth
        layer_found = False
        for res in layer_results_list:
            if res['Top'] <= target_depth_cm + 1e-6 and target_depth_cm < res['Bottom'] - 1e-6 :
                site_result_wp = res['WP']
                site_result_sat = res['SAT']
                site_result_porosity = res['Porosity']
                layer_found = True
                if pd.isna(site_result_wp) or pd.isna(site_result_sat) or pd.isna(site_result_porosity):
                     print(f"    Info: Target depth {target_depth_cm}cm found in layer Top={res['Top']:.1f}-Bottom={res['Bottom']:.1f}, but calculated WP/SAT/Porosity is NaN.")
                break
        if not layer_found:
             print(f"  Warning: Target depth {target_depth_cm}cm not found within any valid layer range for profile {profile_id_int}")

    # Store results for this site and add to cache
    results.append({
        'Site_ID': site_id,
        'WP': site_result_wp,
        'SAT': site_result_sat,
        'Porosity': site_result_porosity
    })
    processed_profiles_cache[profile_id_int] = {
        'WP': site_result_wp,
        'SAT': site_result_sat,
        'Porosity': site_result_porosity
    }


--- Processing Sites ---
Processing Site: ALB_MS (Profile: 1130) - Calculating...
Processing Site: ALB_RF (Profile: 1130) - Using cached results
Processing Site: AMM (Profile: 1281) - Calculating...
Processing Site: AMR (Profile: 1281) - Using cached results
Processing Site: ANK_PT (Profile: 1281) - Using cached results
Processing Site: ASD_MP (Profile: 1050) - Calculating...
Processing Site: BUO (Profile: 1170) - Calculating...
Processing Site: BUW (Profile: 15130) - Calculating...
Processing Site: CAM (Profile: 1050) - Using cached results
Processing Site: DEM (Profile: 1050) - Using cached results
Processing Site: HOC (Profile: 1200) - Calculating...
Processing Site: HOH (Profile: 1200) - Using cached results
Processing Site: ILP_PT (Profile: 1281) - Using cached results
Processing Site: LAW_MOB (Profile: 1130) - Using cached results
Processing Site: LAW_MS (Profile: 1130) - Using cached results
Processing Site: LDC (Profile: 1281) - Using cached results
Processing Site: LDH (Profi

### Step 6: Combine results to the original dataframe

In [11]:
results_df = pd.DataFrame(results)

# Merge results back, ensure we don't duplicate columns if run multiple times
if 'WP' in sites_df.columns: sites_df.drop(columns=['WP'], inplace=True)
if 'SAT' in sites_df.columns: sites_df.drop(columns=['SAT'], inplace=True)
if 'Porosity' in sites_df.columns: sites_df.drop(columns=['Porosity'], inplace=True)

# Use the original sites_df and merge results based on Site_ID
sites_df_final = pd.merge(sites_df, results_df, on=site_id_col, how='left')

#sites_df_final.info()
sites_df_final.head(21)

,Site_ID,Site_Name,Province,BOFEK_2020_Physical Units,Peat_Thickness_2022,Dominant_iProfile,WP,SAT,Porosity
0,ALB_MS,Aldeboam North,Fryslân,1018,143.750000,1130,0.328796,0.718626,0.718626
1,ALB_RF,Aldeboam North,Fryslân,1018,143.250000,1130,0.328796,0.718626,0.718626
2,AMM,Aldeboam Maize,Fryslân,1006,193.500000,1281,0.319729,0.765452,0.765452
3,AMR,Aldeboam Maize,Fryslân,1006,184.500000,1281,0.319729,0.765452,0.765452
4,ANK_PT,Ankeveen,Noord-Holland,1006,-1.000000,1281,0.319729,0.765452,0.765452
5,ASD_MP,Assendelft,Noord-Holland,1001,-1.000000,1050,0.328796,0.718626,0.718626
6,BUO,De Burd,Fryslân,1012,189.500000,1170,0.328796,0.718626,0.718626
7,BUW,De Burd,Fryslân,4002,-1.000000,15130,0.321479,0.591286,0.591286
8,CAM,Capmhyus,Drenthe,1001,160.666667,1050,0.328796,0.718626,0.718626
9,DEM,Demmerik,Utrecht,1001,-1.000000,1050,0.328796,0.718626,0.718626


In [12]:
output_path = "C:/Data_MSc_Thesis/BOFEK/WP_SAT_BOFEK.csv"
sites_df_final.to_csv(output_path, index=False)
print(f"DataFrame successfully saved to {output_path}")

DataFrame successfully saved to C:/Data_MSc_Thesis/BOFEK/WP_SAT_BOFEK.csv
